# 提取所有题目信息

In [23]:
import json
import pandas as pd

In [24]:
LANGUAGE_LIST = ["python", "javascript", "typescript", "java", "c&cpp"]
SUFFIX_MAP = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "c&cpp": "cpp",
    "java": "java"
}


def generate_python_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a python function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a python class , the class signature as below {code_signature}"


def generate_javascript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a javascript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a javascript class , the class signature as below {code_signature}"


def generate_typescript_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a typescript function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a typescript class , the class signature as below {code_signature}"


def generate_ccpp_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a cpp function , the function signature as below {code_signature}"
    elif code_type == "class":
        return f"please write a cpp class , the class signature as below {code_signature}"


def generate_java_prompt(code_signature, code_type):
    if code_type == "method":
        return f"please write a java function , the function signature as below {code_signature},package is org.real.temp, class is Answer"
    elif code_type == "class":
        return f"please write a java class , the function signature as below {code_signature},package is org.real.temp, class is Answer"


LANGUAGE_PROMPT_MAP = {
    "python": generate_python_prompt,
    "javascript": generate_javascript_prompt,
    "typescript": generate_typescript_prompt,
    "c&cpp": generate_ccpp_prompt,
    "java": generate_java_prompt
}

In [26]:
total_question = []
need_question_list = ["python"]
for language in LANGUAGE_LIST:
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    for index, row in data.iterrows():
        task_id = int(row['task_id'])
        code_type = row['code_type']
        original_language = language
        file_path = row['file_path']
        question_type = row['question_type']
        summary = row['task_summary_en']
        temp_data = {
            "task_id": task_id,
            "code_type": code_type,
            "original_language": original_language,
            "file_path": file_path,
            "question_type": question_type,
            "summary": summary, "language_version_list": {
                "python": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "javascript": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "typescript": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "c&cpp": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                },
                "java": {
                    "code_signature": "",
                    "test_code": "",
                    "prompt": "",
                    "addition_info": ""
                }
            }
        }
        for q in need_question_list:
            language_version_list = temp_data["language_version_list"]
            language_info = language_version_list[q]
            dir_path = f"../all/t{task_id}"
            if language == "java":
                signature_path = f"{dir_path}/{language}/Signature.{SUFFIX_MAP[language]}"
            else:
                signature_path = f"{dir_path}/{language}/signature.{SUFFIX_MAP[language]}"
            addition_info = str(row[f"{language}_addition_info"])
            if addition_info == 'nan':
                addition_info = ""
            with open(signature_path, "r", encoding="utf8") as signature_file:
                code_signature = signature_file.read()
            if language == "java":
                test_path = f"{dir_path}/{language}/Tester.{SUFFIX_MAP[language]}"
            else:
                test_path = f"{dir_path}/{language}/test.{SUFFIX_MAP[language]}"
            with open(test_path, "r", encoding="utf8") as test_file:
                code_test = test_file.read()
            prompt = LANGUAGE_PROMPT_MAP[language](code_signature, code_type)
            language_info["code_signature"] = code_signature
            language_info["test_code"] = code_test
            language_info["prompt"] = prompt
            language_info["addition_info"] = addition_info
        total_question.append(temp_data)

with open("./all_question.json", "w", encoding="utf8") as file:
    json_str = json.dumps(total_question)
    file.write(json_str)
    file.flush()

In [5]:
for language in LANGUAGE_LIST:
    # 读取excel表 获取数据信息
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    # 遍历每一行
    question_array = []

    for index, row in data.iterrows():
        task_id = int(row["task_id"])
        code_type = row["code_type"]
        dir_path = f"../all/t{task_id}"
        if language == "java":
            signature_path = f"{dir_path}/{language}/Signature.{SUFFIX_MAP[language]}"
        else:
            signature_path = f"{dir_path}/{language}/signature.{SUFFIX_MAP[language]}"
        addition_info = str(row["addition_info"])
        if addition_info == 'nan':
            addition_info = ""
        with open(signature_path, "r", encoding="utf8") as signature_file:
            code_signature = signature_file.read()
        if language == "java":
            test_path = f"{dir_path}/{language}/Tester.{SUFFIX_MAP[language]}"
        else:
            test_path = f"{dir_path}/{language}/test.{SUFFIX_MAP[language]}"
        with open(test_path, "r", encoding="utf8") as test_file:
            code_test = test_file.read()
        prompt = LANGUAGE_PROMPT_MAP[language](code_signature, code_type)
        question_info = {
            "task_id": task_id,
            "code_type": code_type,
            "code_signature": code_signature,
            "code_language": language,
            "test_code": code_test,
            "prompt": prompt,
            "addition_info": addition_info

        }
        question_array.append(question_info)
        print(f"task_id:{task_id} signature:{len(code_signature)} test:{len(code_test)}")
    with open(f"./question/{language}.json", "w", encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()
    with open(rf"E:\code\code_back\python_project\llm\qa\question\{language}.json", "w",encoding="utf8") as question_file:
        json_str = json.dumps(question_array)
        question_file.write(json_str)
        question_file.flush()

task_id:1 signature:458 test:744
task_id:5 signature:362 test:1591
task_id:6 signature:316 test:661
task_id:7 signature:512 test:1048
task_id:8 signature:513 test:1001
task_id:9 signature:375 test:1022
task_id:11 signature:300 test:1239
task_id:12 signature:597 test:1665
task_id:13 signature:0 test:3520
task_id:17 signature:528 test:1746
task_id:18 signature:278 test:971
task_id:19 signature:319 test:833
task_id:21 signature:501 test:1991
task_id:22 signature:484 test:1570
task_id:23 signature:566 test:1421
task_id:24 signature:267 test:2662
task_id:26 signature:411 test:936
task_id:28 signature:457 test:1399
task_id:31 signature:460 test:918
task_id:33 signature:453 test:2351
task_id:35 signature:486 test:1067
task_id:36 signature:810 test:2157
task_id:37 signature:681 test:1801
task_id:41 signature:346 test:1335
task_id:42 signature:412 test:857
task_id:43 signature:375 test:1524
task_id:44 signature:328 test:2439
task_id:45 signature:575 test:1333
task_id:46 signature:425 test:1747
